---
- 작성자 : 이휘
- 작성일 : 2024-06-04
---

In [1]:
# 데이터 프레임과 계산을 위한 라이브러리
import pandas as pd
import numpy as np

# 차트
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 차트 경고 메세지 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

In [ ]:
# 날짜, 요일, 홈팀, 원정팀, 장소, 관중 수 데이터 불러오기
data_01 = pd.read_csv("./Data/kiwoom_heroes_info(2022~2024).csv")
data_01.head()

In [ ]:
# 속성 확인
data_01.info()

In [ ]:
# 날짜 데이터를 datetime으로 type으규식 로 변환할 수 있게 정사용하여 데이터 변환
data_01['날짜'] = data_01['날짜'].str.replace(r'\([가-힣]\)', '', regex=True)
# 날짜 컬럼을 datetime으로 type 변환
data_01['날짜'] = pd.to_datetime(data_01['날짜'])
data_01.head()

In [ ]:
# 속성 확인
data_01.info()

In [ ]:
# 날짜, 요일, 홈팀, 원정팀, 장소, 관중 수 데이터 불러오기
data_02 = pd.read_csv("./Data/test.csv")
data_02.head()

In [ ]:
# 필요한 데이터 병합하기
kiwoom = pd.concat(
    [
        data_01['날짜'],
        data_01['요일'],
        data_01['원정팀'],
        data_01['관중 수'],
        data_02['시작시간']
    ],
    axis=1
    )
kiwoom

In [ ]:
# 속성 확인
kiwoom.info()

In [ ]:
# 연도, 월 컬럼으로 분리하기
kiwoom['연도'] = kiwoom['날짜'].dt.year
kiwoom

In [ ]:
# 속성 확인
kiwoom.info()

----
### 목표 : 2022~2023년 데이터와 2024년 데이터 분리하기
> 관중 수 예측 모델을 만들어 2022~2023년 데이터를 train 데이터로 학습시키고 <br>
학습한 예측 모델을 가지고 2024 데이터를 test 데이터로 사용하자

In [ ]:
# 2022~2023년 데이터와 2024년 데이터 분리
kiwoom_2022_2023 = \
    kiwoom[(kiwoom['연도'] == 2022) | (kiwoom['연도'] == 2023)]
kiwoom_2024 = kiwoom[kiwoom['연도'] == 2024]
kiwoom_2024 = kiwoom_2024.reset_index(drop=True)

In [ ]:
# 분리 되었는지 확인
kiwoom_2022_2023['연도'].unique()

In [ ]:
# 분리 되었는지 확인
kiwoom_2024['연도'].unique()

In [ ]:
# 스타일
plt.style.use('classic')

kiwoom_2022_2023['관중 수'].plot(
    kind='hist',
    bins=20,
    color='coral',
    figsize=(15,7)
)

plt.title('Historam')
plt.xlabel('Fan')

plt.show()

----
#### 2022~2023년 요일별 관중 수

In [ ]:
sns.boxplot(
    data=kiwoom_2022_2023,
    x='요일',
    y='관중 수'
)
plt.title('KBO 리그 요일별 2022~2023년 요일별 관중 수')
plt.show()

In [ ]:
# # plt.scatter(x, kiwoom_2022_2023['관중 수'], s=area, c=colors, alpha=0.5)
# plt.figure(figsize=(10, 6))
# plt.scatter(kiwoom_2022_2023.index, kiwoom_2022_2023['관중 수'])
# # 그래프 표시
# plt.show()

In [ ]:
# 데이터 불러오기 (kiwoom_2022_2023 DataFrame)
data = kiwoom_2022_2023.groupby('요일')['관중 수'].mean().sort_values()

# 막대 그래프 생성
plt.figure(figsize=(10, 6))
plt.bar(data.index, data.values, color='skyblue')  # 요일 이름 사용
plt.xlabel("요일")
plt.ylabel("평균 관중 수")
plt.ylim(3000,10000)
plt.title("KBO 리그 요일별 평균 관중 수(2022~2023년)")

# 그래프 표시
plt.show()

> 목요일이 가장 평균 관중 수가 가장 낮고, 토요일이 가장 높다. <br>
> 상관 관계를 위해 위의 데이터를 수치화 작업하자 <br>
> 목요일 -> 1<br>
> 수요일 -> 2<br>
> 화요일 -> 3<br>
> 금요일 -> 4<br>
> 일요일 -> 5<br>
> 토요일 -> 6<br>

In [ ]:
# 변환
day_mapping = {
    '목': 1,
    '수': 2,
    '화': 3,
    '금': 4,
    '일': 5,
    '토': 6
}
# 변환 적용
kiwoom_2022_2023['요일'].replace(day_mapping, inplace=True)
kiwoom_2024['요일'].replace(day_mapping, inplace=True)

In [ ]:
kiwoom_2022_2023

In [ ]:
kiwoom_2022_2023[['요일','관중 수']].corr()

In [ ]:
sns.boxplot(
    data=kiwoom_2022_2023,
    x='시작시간',
    y='관중 수',
)
plt.title("KBO 리그 시작시간별 평균 관중 수(2022~2023년)")
plt.show()

### 시작 시간
- 야구 시작시간<br>
- 주말 -> 14:00, 17:00<br>
- 평일 -> 18:30<br>
- 혹서기 한정 -> 18:00(7,8월일 때 17:00 경기를 1시간 미룸)

In [ ]:
kiwoom_2022_2023['시작시간'].unique()

In [ ]:
# 2022~2023년 시작시간이 18:00인 데이터 확인(2건)
kiwoom_2022_2023[kiwoom_2022_2023['시작시간'] == '18:00']

In [ ]:
# 총 2건 밖에 안되므로 17:00 값으로 대체
# 18:00 시작시간 데이터 선택
target_rows = kiwoom_2022_2023[kiwoom_2022_2023['시작시간'] == '18:00']

# 시작시간 변경
kiwoom_2022_2023.loc[target_rows.index, '시작시간'] = '17:00'

In [ ]:
# 시작시간 컬럼 속성 확인
kiwoom_2022_2023['시작시간'].unique()

----
#### 2022~2023년 시작시간별 평균 관중 수

In [ ]:
# 데이터 불러오기 (kiwoom_2022_2023 DataFrame)
data = kiwoom_2022_2023.groupby('시작시간')['관중 수'].mean().sort_values()

# 막대 그래프 생성
plt.figure(figsize=(10, 6))
plt.bar(data.index, data.values, color='skyblue')
plt.xlabel("시작시간")
plt.ylabel("평균 관중 수")
plt.ylim(3000,10000)
plt.title("KBO 리그 시작시간별 평균 관중 수(2022~2023년)")

# 그래프 표시
plt.show()

> 18:30이 가장 평균 관중 수가 가장 낮고, 14:00이 가장 높다. <br>
> 상관 관계를 위해 위의 데이터를 수치화 작업하자 <br>
> 18:30 -> 1<br>
> 17:00 -> 2<br>
> 14:00 -> 3<br>

In [ ]:
# 변환
day_mapping = {
    '18:30': 1,
    '17:00': 2,
    '14:00': 3
}

# 변환 적용
kiwoom_2022_2023['시작시간'].replace(day_mapping, inplace=True)
kiwoom_2024['시작시간'].replace(day_mapping, inplace=True)

In [ ]:
kiwoom_2022_2023

In [ ]:
kiwoom_2022_2023[['관중 수','요일','시작시간']].corr()

----
#### 주중 경기/주말(공휴일) 경기 나누자

In [ ]:
len(kiwoom_2022_2023.loc[(kiwoom_2022_2023['요일'] == 5) | (kiwoom_2022_2023['요일'] == 6)])

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['연도'] == 2022]

> 2022년 공휴일(월, 주말 제외)<br>
05-05, 06-01, 09-09

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2022-06-01']

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2022-09-09']

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['연도'] == 2023]

> 2022년 공휴일(월, 주말 제외)<br>
05-05, 06-06, 08-15, 09-28, 09-29

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2023-05-05']

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2023-06-06']

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2023-09-28']

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2023-09-29']

In [ ]:
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2022-06-01']
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2022-09-09']
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2023-05-05']
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2023-06-06']
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2023-09-28']
kiwoom_2022_2023[kiwoom_2022_2023['날짜'] == '2023-09-29']

In [ ]:
kiwoom_2022_2023['주중/주말(공휴일)'] = ''

In [ ]:
holidays = ['2022-06-01', '2022-09-09', '2023-05-05', '2023-06-06', '2023-09-28', '2023-09-29']

for holiday in holidays:
    kiwoom_2022_2023.loc[kiwoom_2022_2023['날짜'] == holiday, '주중/주말(공휴일)'] = 1

In [ ]:
kiwoom_2022_2023.loc[kiwoom_2022_2023['요일'] == 5, '주중/주말(공휴일)'] = 1
kiwoom_2022_2023

In [ ]:
kiwoom_2022_2023.loc[kiwoom_2022_2023['요일'] == 6, '주중/주말(공휴일)'] = 1
kiwoom_2022_2023

In [ ]:
kiwoom_2022_2023.loc[kiwoom_2022_2023['주중/주말(공휴일)'] != 1, '주중/주말(공휴일)'] = 0
kiwoom_2022_2023

In [ ]:
kiwoom_2022_2023['주중/주말(공휴일)'].isnull().sum()

In [ ]:
kiwoom_2022_2023['주중/주말(공휴일)'] = kiwoom_2022_2023['주중/주말(공휴일)'].astype('int')
kiwoom_2022_2023

In [ ]:
kiwoom_2022_2023[['관중 수','주중/주말(공휴일)','요일','시작시간']].corr()

In [ ]:
kiwoom_2024

In [ ]:
kiwoom_2024.loc[kiwoom_2024['요일'] == 5, '주중/주말(공휴일)'] = 1
kiwoom_2024

In [ ]:
kiwoom_2024.loc[kiwoom_2024['요일'] == 6, '주중/주말(공휴일)'] = 1
kiwoom_2024

In [ ]:
kiwoom_2024.loc[kiwoom_2024['주중/주말(공휴일)'] != 1, '주중/주말(공휴일)'] = 0
kiwoom_2024

In [ ]:
kiwoom_2024['주중/주말(공휴일)'] = kiwoom_2024['주중/주말(공휴일)'].astype('int')
kiwoom_2024

----
#### 2022~2023년 원정팀별 평균 관중 수

In [ ]:
# 데이터 불러오기 (kiwoom_2022_2023 DataFrame)
data = kiwoom_2022_2023.groupby('원정팀')['관중 수'].mean().sort_values()

# 막대 그래프 생성
plt.figure(figsize=(10, 6))
plt.bar(data.index, data.values, color='skyblue')  # 요일 이름 사용
plt.xlabel("원정팀")
plt.ylabel("평균 관중 수")
plt.ylim(3000,10000)
plt.title("KBO 리그 원정팀별 평균 관중 수(2022~2023년)")

# 그래프 표시
plt.show()

> KT 경기가 가장 평균 관중 수가 가장 낮고, 롯데 경기가 가장 높다. <br>
> 상관 관계를 위해 위의 데이터를 수치화 작업하자 <br>
> KT -> 1<br>
> NC -> 2<br>
> 두산 -> 3<br>
> 삼성 -> 4<br>
> LG -> 5<br>
> SSG -> 6<br>
> 한화 -> 7<br>
> KIA -> 8<br>
> 롯데 -> 9<br>

In [ ]:
# 변환 맵 정의
away_mapping = {
    'KT': 1,
    'NC': 2,
    '두산': 3,
    '삼성': 4,
    'LG': 5,
    'SSG': 6,
    '한화': 7,
    'KIA': 8,
    '롯데': 9
}

# 변환 적용
kiwoom_2022_2023['원정팀'].replace(away_mapping, inplace=True)
kiwoom_2024['원정팀'].replace(away_mapping, inplace=True)

In [ ]:
kiwoom_2022_2023

In [ ]:
# column 이름 문자열로 바꾸기
kiwoom_2022_2023.columns = kiwoom_2022_2023.columns.astype(str)
kiwoom_2024.columns = kiwoom_2024.columns.astype(str)

In [ ]:
kiwoom_2022_2023 = kiwoom_2022_2023.drop(['날짜','연도','월'], axis=1)
kiwoom_2024 = kiwoom_2024.drop(['날짜','연도','월'], axis=1)

In [ ]:
# # One-Hot Encoding 작업
# # 원정팀
# away_encoding = pd.get_dummies(kiwoom_2022_2023['원정팀'])
# kiwoom_2022_2023 = kiwoom_2022_2023.join(away_encoding)

In [ ]:
# # One-Hot Encoding 작업
# # 원정팀
# away_encoding = pd.get_dummies(kiwoom_2024['원정팀'])
# kiwoom_2024 = kiwoom_2024.join(away_encoding)

In [ ]:
kiwoom_2022_2023.head()

In [ ]:
kiwoom_2024.head()

In [ ]:
# 2022-2023년 데이터 정리
# One-Hot Encoding 부분을 boolean에서 int로 변환
kiwoom_2022_2023.iloc[:,8:] = kiwoom_2022_2023.iloc[:,8:].astype('int')
# column 이름 문자열로 바꾸기
kiwoom_2022_2023.columns = kiwoom_2022_2023.columns.astype(str)
# Target인 관중 수를 제외한 나머지 원본 Feature 부분 제외하고 데이터 불러오기
kiwoom_2022_2023 = pd.concat([kiwoom_2022_2023['관중 수'],kiwoom_2022_2023['요일'],kiwoom_2022_2023['시작시간'],kiwoom_2022_2023['주중/주말(공휴일)'],kiwoom_2022_2023.iloc[:,8:]],axis=1)
kiwoom_2022_2023.head(10)

In [ ]:
# 2024년 데이터 정리
# One-Hot Encoding 부분을 boolean에서 int로 변환
kiwoom_2024.iloc[:,8:] = kiwoom_2024.iloc[:,8:].astype('int')
# column 이름 문자열로 바꾸기
kiwoom_2024.columns = kiwoom_2024.columns.astype(str)
# Target인 관중 수를 제외한 나머지 원본 Feature 부분 제외하고 데이터 불러오기
kiwoom_2024 = pd.concat([kiwoom_2024['관중 수'],kiwoom_2024['요일'],kiwoom_2024['시작시간'],kiwoom_2024['주중/주말(공휴일)'],kiwoom_2024.iloc[:,8:]],axis=1)
kiwoom_2024.head(10)

In [ ]:
# 분석한 Target과 Feature 열만 가져오기
kiwoom_2022_2023 = \
    pd.concat(
        [
            kiwoom_2022_2023['관중 수'],
            kiwoom_2022_2023['요일'],
            kiwoom_2022_2023['시작시간']
        ],
        axis=1
        )
kiwoom_2022_2023.head(10)

In [ ]:
kiwoom_2022_2023.corr()

In [ ]:
# 2022-2023 데이터 임시 저장
# kiwoom_2022_2023.to_csv('./Data/2022-2023_One-hot.csv')

----
## 관중 수 범위 찾기

In [ ]:
# k-mean로 관중 수 범위 나눌 k 값 찾기
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 데이터 전처리: 타깃 컬럼과 피쳐 컬럼 분리
features = kiwoom_2022_2023.drop(columns=['관중 수'])
target = kiwoom_2022_2023['관중 수']

attendance_shape = np.array(target).reshape(-1,1)

# 특성 표준화
scaler = StandardScaler()
features_scaled = scaler.fit_transform(attendance_shape)

# 엘보우 방법을 사용하여 최적의 클러스터 수 찾기
sse = []
k_range = range(3, 15)  # 3부터 15까지의 k 값에 대해 확인

for k in k_range:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(attendance_shape)
    sse.append(kmeans.inertia_)

# 엘보우 그래프 그리기
plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of squared distances (SSE)')
plt.title('Elbow Method For Optimal k')
plt.grid(True)
plt.show()


> k값의 최적은 5이므로, 관중 수는 5구간으로 나누는 것이 적합.

In [ ]:
# K-means 클러스터링
kmeans = KMeans(n_clusters=5)
clusters = kmeans.fit_predict(attendance_shape)
clusters

# 클러스터 라벨을 데이터프레임에 추가
kiwoom_2022_2023['Cluster'] = clusters

# 각 클러스터의 관중 수 범위 계산
cluster_ranges = {}

for cluster in range(5):
    cluster_data = \
        kiwoom_2022_2023[kiwoom_2022_2023['Cluster'] == cluster]
    min_attendance = \
        cluster_data['관중 수'].min()
    max_attendance = \
        cluster_data['관중 수'].max()
    cluster_ranges[cluster] = (min_attendance, max_attendance)

# 클러스터의 관중 수 범위를 정렬
sorted_cluster_ranges = \
sorted(cluster_ranges.items(), key=lambda x: x[1])
sorted_cluster_ranges

> 관중 수를 클러스터링으로 분류 결과 <br>  
~3500 -> 1 <br>
3500~6000 -> 2 <br>
6000~9000 -> 3 <br>
9000~13000 -> 4 <br>
13000~ -> 5 <br>

In [ ]:
# 규칙에 따라 관중 수를 분류하여 새로운 컬럼에 할당
kiwoom_2022_2023['관중수 분류'] = \
    pd.cut(kiwoom_2022_2023['관중 수'], 
    bins=[0, 3500, 6000, 9000, 13000, float('inf')], 
    labels=[1, 2, 3, 4, 5], 
    right=False)

In [ ]:
kiwoom_2022_2023 = \
kiwoom_2022_2023.drop(['Cluster','관중 수'], axis=1)
kiwoom_2022_2023

In [ ]:
# 규칙에 따라 관중 수를 분류하여 새로운 컬럼에 할당
kiwoom_2024['관중수 분류'] = pd.cut(kiwoom_2024['관중 수'], 
                                    bins=[0, 3500, 6000, 9000, 13000, float('inf')], 
                                    labels=[1, 2, 3, 4, 5], 
                                    right=False)

In [ ]:
kiwoom_2024 = kiwoom_2024.drop(['관중 수'], axis=1)
kiwoom_2024

In [ ]:
# plt.scatter(
#     x=kiwoom_2022_2023.index,
#     y=kiwoom_2022_2023['관중 수'],
#     c=clusters
# )

----
# 예측


In [ ]:
# 각 column별 상관계수 확인
kiwoom_2022_2023.corr()

In [ ]:
# Randomforest 회귀모델로 정확도율 구하기
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 데이터 준비 (x는 피쳐, y는 타겟)
x = kiwoom_2022_2023.drop(['관중 수'], axis=1)
y = kiwoom_2022_2023['관중 수']

# 학습 데이터와 테스트 데이터로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# 랜덤 포레스트 회귀모델 학습
rf_regressor = RandomForestRegressor()
rf_regressor.fit(x_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf_regressor.predict(x_test)

# 오차 계산 (평균 제곱 오차)
mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error:", mse)

# MAPE 계산 함수 정의
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    print(y_pred.round(0))
    print(y_true)
    # return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return (1 - np.mean(np.abs((y_true - y_pred) / y_true))) * 100


# MAPE 계산
mape = mean_absolute_percentage_error(y_test, y_pred)
print("정확도율:", mape.round(0), "%")

----
### 랜덤포레스트 클러스터링 예측

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

----
### Train, test 구분

In [ ]:
# 데이터 준비 (x는 피쳐, y는 타겟)
x = kiwoom_2022_2023.drop(['관중수 분류'], axis=1)
y = kiwoom_2022_2023['관중수 분류']

# 실제 데이터 준비
test_x = kiwoom_2024.drop(['관중수 분류'], axis=1)
test_y = kiwoom_2024['관중수 분류']

In [ ]:
def cross_validation(classifier, features, labels):
    cv_scores = []
    for i in range(10):
        scores = cross_val_score(
            classifier, # 함수
            features,
            labels,
            cv=10,
            scoring='accuracy',
            n_jobs=-1
        )
        cv_scores.append(round(scores.mean()*100,0))
    return cv_scores

In [ ]:
# 랜덤포레스트
cross_validation(RandomForestClassifier(), x, y)

In [ ]:
# 랜덤포레스트
cross_validation(SVC(), x, y)

In [ ]:
# 의사결정 나무
cross_validation(tree.DecisionTreeClassifier(), x, y)

In [ ]:
# KNN
cross_validation(KNeighborsClassifier(n_neighbors=5), x, y)

In [ ]:
# 학습 데이터와 테스트 데이터로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# 랜덤 포레스트 분류모델 학습
svm_clf = SVC()
# rf_clf_01 = RandomForestClassifier()
svm_clf.fit(x_train, y_train)
# rf_clf.fit(x, y)

# 테스트 데이터에 대한 예측
# y_pred = rf_clf.predict(x_test)
# y_pred_t = rf_clf.predict(test_x)
scores = cross_val_score(svm_clf, x_test, y_test, cv=5)

print("교차 검증 점수:", scores*100)
print(f"평균 교차 검증 점수:{round(scores.mean()*100)}%")    

# 정확도 계산
# accuracy = accuracy_score(y_test, y_pred)
# accuracy_t = accuracy_score(test_y, y_pred_t)
# print(f"학습예측율 : {scores * 100:.2f}%({i+1}회)")
# print(f"학습예측율 : {accuracy * 100:.2f}%({i+1}회)")
# print(f"실제예측율 : {accuracy * 100:.2f}%({i+1}회)")

In [ ]:
# Train용 함수
def trainML(model):
    # 5번 반복하여 학습 및 평가
    for i in range(5):

        # 학습 데이터와 테스트 데이터로 분리
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        
        # 랜덤 포레스트 분류모델 학습
        dtree = model
        dtree.fit(x_train, y_train)

        # 테스트 데이터에 대한 예측
        y_pred = dtree.predict(x_test)

        # 정확도 계산
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        print(f"예측율 : {accuracy * 100:.2f}%({i+1}회)")

----
### 랜덤포레스트

In [ ]:
trainML(RandomForestClassifier())

----
### 의사결정나무

----
### KNN

----
### SVM

In [ ]:
trainML(SVC(probability=True).fit(x, y))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier

In [ ]:
# 학습 데이터와 테스트 데이터로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# 특성 만들기
poly = PolynomialFeatures(degree=7, include_bias=False)
poly.fit(x_train)
train_poly = poly.transform(x_train)
test_poly = poly.transform(x_test)

# 데이터 스케일링
ss = StandardScaler()
ss.fit(x_train)
train_scaled = ss.transform(x_train)
test_scaled = ss.transform(x_test)

In [ ]:
# 분류기 학습
clf = RidgeClassifier()
clf.fit(train_scaled, y_train)

# 교차 검증 수행 및 평가
scores = cross_val_score(clf, x_test, y_test, cv=5)
print("교차 검증 점수:", scores*100)
print("평균 교차 검증 점수:", scores.mean()*100,'%')

In [ ]:
# 분류기 학습
kncf = KNeighborsClassifier(n_neighbors=5)
kncf.fit(train_scaled, y_train)

# 교차 검증 수행 및 평가
scores = cross_val_score(kncf, x_test, y_test, cv=5)
print("교차 검증 점수:", scores*100)
print("평균 교차 검증 점수:", scores.mean()*100,'%')

In [ ]:
# Feature
kiwoom_2022_2023_input = kiwoom_2022_2023.drop(['관중수 분류'], axis=1).to_numpy()
kiwoom_2022_2023_input[:5]

In [ ]:
# Target Data
kiwoom_2022_2023_target = kiwoom_2022_2023['관중수 분류'].to_numpy()
kiwoom_2022_2023_target[:5]

In [ ]:
# Train과 Test 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
    train_test_split(kiwoom_2022_2023_input, kiwoom_2022_2023_target, stratify=kiwoom_2022_2023_target, random_state=42)

In [ ]:
# 표준화 작업
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sc = SGDClassifier(
    loss='log', # 로지스틱 손실함수
    # loss='log_loss', # 로지스틱 손실함수
    max_iter=10, # 반복 횟수
    random_state=42
    )

In [ ]:
sc.fit(train_scaled, train_target)

In [ ]:
print('train score :', sc.score(train_scaled, train_target))
print('test score :', sc.score(test_scaled, test_target))

In [ ]:
sc.partial_fit(train_scaled, train_target)
print('train score :', sc.score(train_scaled, train_target))
print('test score :', sc.score(test_scaled, test_target))

In [ ]:
import numpy as np
sc = SGDClassifier(loss='log', random_state=42)
# sc = SGDClassifier(loss='log_loss', random_state=42)
train_score = []
test_score = []

classes = np.unique(train_target)
classes

In [ ]:
for _ in range(0,300):
    sc.partial_fit(train_scaled, train_target, classes=classes)
    train_score.append(sc.score(train_scaled, train_target))
    test_score.append(sc.score(test_scaled, test_target))

In [ ]:
# 그래프로
import matplotlib.pyplot as plt

plt.plot(train_score)
plt.plot(test_score)

plt.show()

In [ ]:
# 새롭게 시작
sc = SGDClassifier(
    loss='log',
    # loss='log_loss',
    max_iter=100,
    tol=None # 조기종료 시점이나 현재는 정지기준이 없고 반복기간동안의 최적의 손실값보다 큰 경우 조기 종료.
)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

In [ ]:
# 새롭게 시작
sc = SGDClassifier(
    loss='hinge', # SVM
    # loss='log_loss',
    max_iter=100,
    tol=1.0e-2 # 기본값.
)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

In [ ]:
# 새롭게 시작
sc = SGDClassifier(
    loss='perceptron', # SVM
    # loss='log_loss',
    max_iter=100,
    tol=1.0e-2 # 기본값.
)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))